# **1. Getting Started**
___

## **1.1 Installing dependencies**

In [ ]:
!pip install evaluate pandas sentence_transformers torch tqdm transformers bert_score rouge_score peft datasets numpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.4 MB/s eta 0:00:00
   ━━━

# **2. Data Preprocessing**

## **The datasets**

1. *genZ_slangs(1.7k_rows)_edited* (created by creating a context column from the genz_slang_dataset using ChatGPT)

2. *genz_slang_dataset* (from https://huggingface.co/datasets/MLBtrio/genz-slang-dataset)

## **CSV curation/conversion into fine-tuning format**

Adds the context column from the *genZ_slangs(1.7k_rows)_edited* into the *genz_slang_dataset*, to create the *genz_finetune_combined_dataset* (which is formatted for the finetuning)

genz_finetune_combined_dataset is saved in *"dataset/"* folder


In [ ]:
import pandas as pd

# === Load Datasets ===

df1 = pd.read_csv("dataset/genZ_slangs(1.7k_rows)_edited.csv")
df2 = pd.read_csv("dataset/genz_slang_dataset.csv")

# === Clean column names ===
df1.columns = df1.columns.str.strip().str.lower()
df2.columns = df2.columns.str.strip().str.lower()

# === Sanity check: align by index and length ===
min_len = min(len(df1), len(df2))
df1 = df1.head(min_len).reset_index(drop=True)
df2 = df2.head(min_len).reset_index(drop=True)

# === Combine example + description + context from df2, and target from df1 ===
combined_data = []
for i in range(min_len):
    example = str(df2.loc[i, "example"]).strip()
    description = str(df2.loc[i, "description"]).strip()
    context = str(df2.loc[i, "context"]).strip()
    target = str(df1.loc[i, "proper english"]).strip()

    if example and target:
        input_text = (
            f"Paraphrase this Gen-Z sentence into formal English:\n"
            f"Example: {example}\n"
            f"Description: {description}\n"
            f"Context: {context}"
        )
        combined_data.append({"input": input_text, "target": target})

# === Save to CSV ===
combined_df = pd.DataFrame(combined_data)
output_path = "dataset/genz_finetune_combined_dataset.csv"
combined_df.to_csv(output_path, index=False)
print(f"Combined dataset saved to: {output_path}")

# **3. Creating the Evaluation Methods**
___


Mean Cosine Similarity : The average cosine similarity of all preds<=>refs\
Mean METEOR : The average METEOR score of all preds<=>refs\
Mean Bert Precision : The average BERT precision of all preds<=>refs\
Mean Bert Recall : The average BERT recall of all preds<=>refs\
Mean Bert f1 : The average BERT f1 of all preds<=>refs\
Mean Rouge-1 : The average Rouge-1 (1-gram) of all preds<=>refs\
Mean Rouge-2 : The average Rouge-2 (2-gram) of all preds<=>refs\
Mean Rouge-L : The average Rouge-L (Longest common substring) of all preds<=>refs\
Geometric Mean BLEU : The geometric mean of all BLEU-n (1 to bleu_n) of all preds<=>refs\
Mean BLEU-n : The average BLEU-n of all preds<=>refs\

In [ ]:
from sentence_transformers import SentenceTransformer, util
from evaluate import load

# Initialise evaluation objects
embedder = SentenceTransformer('all-MiniLM-L6-v2')
bleu = load("bleu")
meteor = load("meteor")
bertscore = load("bertscore")
rouge = load("rouge")

def cosine_similarity(preds, refs):
    pred_emb = embedder.encode(preds, convert_to_tensor=True)
    ref_emb = embedder.encode(refs, convert_to_tensor=True)
    return util.cos_sim(pred_emb, ref_emb).diagonal().tolist()

def bleu_score(preds, refs, n=2):
    results = bleu.compute(predictions=preds, references=refs, smooth=True, max_order=n)
    return results

def meteor_score(preds, refs):
    results = meteor.compute(predictions=preds, references=refs)
    return results

def bert_score(preds, refs):
    results = bertscore.compute(predictions=preds, references=refs, lang='en', model_type="distilbert-base-uncased")
    return results

def rouge_score(preds, refs):
    results = rouge.compute(predictions=preds, references=refs)
    return results

def eval_model(model_name, data, bleu_n=2):
    # data is a df with preds and refs column
    # model_name is a str
    # bleu_n is an int, default 2
    length = len(data)
    cos_sim_eval_score = cosine_similarity(preds=data['preds'], refs=data['refs'])
    bleu_eval_score = bleu_score(preds=data['preds'], refs=data['refs'], n=bleu_n)
    METEOR_eval_score = meteor_score(preds=data['preds'], refs=data['refs'])
    Bert_eval_score = bert_score(preds=data['preds'], refs=data['refs'])
    rouge_eval_score = rouge_score(preds=data['preds'], refs=data['refs'])

    eval_scores = {
        'Model name': model_name,
        'Mean Cosine Similarity': sum(cos_sim_eval_score)/length,
        'Mean METEOR' : METEOR_eval_score['meteor'],
        # 'Mean Bert Precision' : sum(Bert_eval_score['precision'])/length,
        # 'Mean Bert Recall' : sum(Bert_eval_score['recall'])/length,
        'Mean Bert f1' : sum(Bert_eval_score['f1'])/length,
        # 'Mean Rouge-1' : rouge_eval_score['rouge1'],
        # 'Mean Rouge-2' : rouge_eval_score['rouge2'],
        'Mean Rouge-L' : rouge_eval_score['rougeL'],
        'Geometric Mean BLEU' : bleu_eval_score['bleu'],
    }

    for i in range(bleu_n):
        eval_scores[f'Mean BLEU-{i+1}'] = bleu_eval_score['precisions'][i]

    results = pd.DataFrame([eval_scores])
    return results


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# **4. Zero-Shot Testing**

___

Evaluating the base models' capabilities to understand gen-z slang (testing against *genz_finetune_combined_dataset*)

## **4.1 Loading the dataset**

In [ ]:
# Zero-Shot Evaluation of Gen-Z to Formal English Translation

## Load Required Libraries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm import tqdm

## Load Your Dataset
# Assuming your dataset is a CSV file with columns: 'Slang', 'Description', 'Example', 'Proper English'
df = pd.read_csv("dataset/genz_finetune_combined_dataset.csv")

In [ ]:
print(df.columns.tolist())

['Slang', 'Description', 'Example', 'Proper English']


## **4.2 Creating the method and running inference**

In [ ]:
## Zero-Shot Inference Function
def evaluate_model(model_name, df, limit=100):
    print(f"\nEvaluating: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    model.eval()

    # preds, refs, sims = [], [], []
    preds, refs = [], []
    for i, row in tqdm(df.head(limit).iterrows(), total=limit):
        prompt = row['input']
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
        with torch.no_grad():
            output = model.generate(**inputs, max_length=60)
        pred = tokenizer.decode(output[0], skip_special_tokens=True)
        preds.append(pred)
        refs.append(row["target"])

    # Preparing for eval_model
    data = {
        'refs': refs,
        'preds': preds
    }
    data = pd.DataFrame(data)

    eval_results = eval_model(model_name=model_name, data=data, bleu_n=2)

    return eval_results

## Evaluate Multiple Models
models_to_test = [
    "google/flan-t5-base",
    "google/flan-t5-large",
    "google/t5-v1_1-base",
    "facebook/bart-large"
]

results = []
for model_name in models_to_test:
    result = evaluate_model(model_name, df)
    results.append(result)

summary = pd.concat(results, ignore_index=True)
print("\n📊 Summary of Results:")
print(summary)


Evaluating: google/flan-t5-base


100%|██████████| 100/100 [04:12<00:00,  2.53s/it]


Average Semantic Similarity: 0.4408

Evaluating: google/flan-t5-large


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]


100%|██████████| 100/100 [20:02<00:00, 12.02s/it]


Average Semantic Similarity: 0.3745

📊 Summary of Results:
                  Model  Avg Semantic Similarity
0   google/flan-t5-base                 0.440839
1  google/flan-t5-large                 0.461683
2   google/t5-v1_1-base                 0.144678
3   facebook/bart-large                 0.374488


# **5. Finetuning**

___

## **5.1 Dataset Preparation**

Cleaning and splitting the dataset

In [ ]:
# Fine-Tuning FLAN-T5 on Gen-Z to Formal English Translation

## Import Libraries
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType

## Load and Prepare Dataset
df = pd.read_csv("dataset/genz_finetune_combined_dataset.csv")

# Strip and clean
df.columns = df.columns.str.strip()
df = df.dropna(subset=["input", "target"])
df = df[df["input"].str.strip() != ""]
df = df[df["target"].str.strip() != ""]

# Keep only relevant columns
df = df[["input", "target"]]  # <- THIS is important!

# Convert safely
dataset = Dataset.from_pandas(df)

# Train/Validation split
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00

## **5.3 Finetuning the model on the *genz_finetune_combined_dataset***

Running inference and evaluating the model's performance at every epoch

In [ ]:
import numpy as np

## Tokenization
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(example):
    input_text = str(example['input']).strip()
    target_text = str(example['target']).strip()

    if input_text == "" or target_text == "":
        return {}

    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=300)
    labels = tokenizer(target_text, truncation=True, padding="max_length", max_length=300)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess, batched=False).filter(lambda x: "labels" in x)
tokenized_val = val_dataset.map(preprocess, batched=False).filter(lambda x: "labels" in x)

## Load Model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q", "k", "v"],
    lora_dropout=0.1,
    task_type=TaskType.SEQ_2_SEQ_LM )

model = get_peft_model(model, lora_config)


## Metric Functions
# meteor = load("meteor")
# bertscore = load("bertscore")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100s in labels for proper decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Ensure preds are a NumPy array of int32
    preds = np.array(preds).astype(np.int32)

    # Hard clip values to valid token range
    preds = np.clip(preds, 0, tokenizer.vocab_size - 1)
    labels = np.clip(labels, 0, tokenizer.vocab_size - 1)

    # Defensive decode with try/except to avoid corrupt entries
    try:
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    except OverflowError as e:
        print("⚠️ Skipping decoding due to invalid token ids")
        return {
            "meteor": 0.0,
            "bertscore_f1": 0.0
        }

    # Strip and filter empty ones
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    paired = [(p, l) for p, l in zip(decoded_preds, decoded_labels) if p and l]
    if not paired:
        return {
            "meteor": 0.0,
            "bertscore_f1": 0.0
        }

    filtered_preds, filtered_labels = zip(*paired)

    # result = {}
    # result.update(meteor.compute(predictions=filtered_preds, references=filtered_labels))
    # result["bertscore_f1"] = np.mean(
    #     bertscore.compute(predictions=filtered_preds, references=filtered_labels, lang="en")["f1"]
    # )

    # return {
    #     "meteor": result["meteor"],
    #     "bertscore_f1": result["bertscore_f1"],
    # }

    bleu_n = 2
    bleu_eval = bleu_score(preds=filtered_preds, refs=filtered_labels, n=bleu_n)
    meteor_eval = meteor_score(preds=filtered_preds, refs=filtered_labels)
    bert_eval = bert_score(preds=filtered_preds, refs=filtered_labels)
    rouge_eval = rouge_score(preds=filtered_preds, refs=filtered_labels)
    cos_sim_eval = cosine_similarity(preds=filtered_preds, refs=filtered_labels)
    length = len(filtered_preds)

    # Package results
    result = {
        "Mean Cosine Similarity": sum(cos_sim_eval) / length,
        "Mean METEOR": meteor_eval["meteor"],
        "Mean Bert f1": sum(bert_eval["f1"]) / length,
        "Mean Rouge-L": rouge_eval["rougeL"],
        "Geometric Mean BLEU": bleu_eval["bleu"],
    }

    for i in range(bleu_n):
        result[f"Mean BLEU-{i+1}"] = bleu_eval["precisions"][i]

    return result

## Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan-t5-genz",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True
)

## Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

## Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

## Model Trainer
trainer.train()

model = model.merge_and_unload()

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

# **6. Saving the model weights after finetuning**

___

In [ ]:
model.save_pretrained("model/flan-t5-genz-final")
tokenizer.save_pretrained("model/flan-t5-genz-final")

('/content/drive/MyDrive/Colab Notebooks/NLP/fine-tuned models/flan-t5-genz-final/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/fine-tuned models/flan-t5-genz-final/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/fine-tuned models/flan-t5-genz-final/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/NLP/fine-tuned models/flan-t5-genz-final/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/NLP/fine-tuned models/flan-t5-genz-final/tokenizer.json')

# **7. Running inference on the finetuned model**
---

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd

# === Load Fine-Tuned Model ===
model_path = "model/flan-t5-genz-final" 

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# === Load Input Prompts for Testing ===
df = pd.read_csv("dataset/genz_finetune_combined_dataset.csv")
test_inputs = df['input'].dropna().sample(5).tolist()

# === Run Inference ===
for inp in test_inputs:
    encoded = tokenizer(inp, return_tensors="pt", truncation=True, padding=True).to(device)
    output = model.generate(**encoded, max_length=60)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    print(f"Prompt: {inp}")
    print(f"Output: {decoded}\\n")


Prompt: Translate this Gen-Z sentence into formal English:
Example: You got double sixes, NR!
Description: Nice roll
Context: A compliment used in dice games when someone gets a good result.
Output: :good\n
Prompt: Translate this Gen-Z sentence into formal English:
Example: BRB, BAK in 5.
Description: Back at keyboard
Context: Used to inform someone that you are back at your computer after being away.
Output: \n
Prompt: Translate this Gen-Z sentence into formal English:
Example: The letter had SWALK written on the envelope.
Description: Sealed with a loving kiss
Context: A more affectionate version of SWAK, often used in love letters.
Output: :SALK written on the envelope.\n
Prompt: Translate this Gen-Z sentence into formal English:
Example: I have such FOMO, everyone’s going to that concert except me.
Description: Fear of missing out
Context: Refers to the feeling of anxiety or regret over missing an event or experience.
Output: \n
Prompt: Translate this Gen-Z sentence into formal Eng

# **Streamlit for visualisation**

___

#### **Installing dependencies**

In [ ]:
!pip install -q streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00


## **Streamlit**

Download the finetuned model from the link https://drive.google.com/drive/folders/1ml4hktQZ0z-tdL0bhyCceqL5F25i1d5a?usp=sharing

Ensure the downloaded finetuned model is saved in the *"model/"* folder

In [ ]:
%%writefile app.py
import streamlit as st

@st.cache_resource
def load_model():
  model_path = "model/flan-t5-genz-final"
  tokenizer = AutoTokenizer.from_pretrained(model_path)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")
  model.eval()
  return model, tokenizer

def translate_genz(model,tokenizer, text):
  prompt = f"Paraphrase this Gen-Z sentence into formal English:\n{text}"
  inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to(model.device)
  output = model.generate(**inputs, max_length=100)
  return tokenizer.decode(output[0], skip_special_tokens=True)

if 'model_loaded' not in st.session_state:
    st.session_state.model_loaded = False

st.title("Gen-Z Translator")
prompt = st.text_input("Enter a Gen-Z sentence")

if st.button("Translate"):
    if not st.session_state.model_loaded:
        st.session_state.model, st.session_state.tokenizer = load_model()
        st.session_state.model_loaded = True
    if st.session_state.model_loaded:
        translation = translate_genz(st.session_state.model, st.session_state.tokenizer, prompt)
        st.write(translation)


Writing app.py


## **Running Streamlit**

Please put your own auth_token

Instructions for how to get the auth_token : https://ngrok.com/docs/agent/

Click on the first link (left) to launch the Streamlit client

In [ ]:
from pyngrok import ngrok

auth_token = 'Your auth_token' # Replace with your own auth_token

ngrok.set_auth_token(auth_token)

!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(addr = '80')
print(url)

nohup: appending output to 'nohup.out'
NgrokTunnel: "https://661a-35-230-54-122.ngrok-free.app" -> "http://localhost:80"
